## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Clyde River,CA,2022-07-22 03:17:41,70.4692,-68.5914,37.63,100,100,5.75,light rain
1,1,Qaanaaq,GL,2022-07-22 03:17:41,77.4840,-69.3632,44.40,97,100,6.91,moderate rain
2,2,East London,ZA,2022-07-22 03:17:42,-33.0153,27.9116,54.54,87,0,6.91,clear sky
3,3,Dicabisagan,PH,2022-07-22 03:17:42,17.0818,122.4157,88.41,66,7,5.46,clear sky
4,4,Black River,JM,2022-07-22 03:17:43,18.0264,-77.8487,79.57,78,7,4.76,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp =float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
3,3,Dicabisagan,PH,2022-07-22 03:17:42,17.0818,122.4157,88.41,66,7,5.46,clear sky
4,4,Black River,JM,2022-07-22 03:17:43,18.0264,-77.8487,79.57,78,7,4.76,clear sky
7,7,Hithadhoo,MV,2022-07-22 03:17:43,-0.6000,73.0833,77.94,87,100,13.53,light rain
9,9,Verkh-Usugli,RU,2022-07-22 03:17:44,52.7000,115.1667,79.38,45,53,5.97,broken clouds
10,10,Seoul,KR,2022-07-22 03:15:11,37.5683,126.9778,83.77,65,75,5.75,broken clouds
11,11,Kijang,KR,2022-07-22 03:17:45,35.2442,129.2139,87.53,70,38,3.98,scattered clouds
14,14,Albany,US,2022-07-22 03:17:25,42.6001,-73.9662,75.58,74,0,4.29,clear sky
17,17,Katsuura,JP,2022-07-22 03:17:47,35.1333,140.3000,82.42,88,45,17.56,scattered clouds
18,18,Kualakapuas,ID,2022-07-22 03:17:47,-3.0091,114.3876,86.52,67,98,4.18,overcast clouds
19,19,Faanui,PF,2022-07-22 03:17:48,-16.4833,-151.7500,77.34,73,16,10.63,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Date                   249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Weather Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Dicabisagan,PH,88.41,clear sky,17.0818,122.4157,
4,Black River,JM,79.57,clear sky,18.0264,-77.8487,
7,Hithadhoo,MV,77.94,light rain,-0.6000,73.0833,
9,Verkh-Usugli,RU,79.38,broken clouds,52.7000,115.1667,
10,Seoul,KR,83.77,broken clouds,37.5683,126.9778,
11,Kijang,KR,87.53,scattered clouds,35.2442,129.2139,
14,Albany,US,75.58,clear sky,42.6001,-73.9662,
17,Katsuura,JP,82.42,scattered clouds,35.1333,140.3000,
18,Kualakapuas,ID,86.52,overcast clouds,-3.0091,114.3876,
19,Faanui,PF,77.34,few clouds,-16.4833,-151.7500,


In [8]:
import requests

In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ''].index, inplace = False)
clean_hotel_df.shape

(238, 7)

In [26]:
# 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
3,Dicabisagan,PH,88.41,clear sky,17.0818,122.4157,Nancy Montanes Bernardo
4,Black River,JM,79.57,clear sky,18.0264,-77.8487,Idlers' Rest Beach Hotel
7,Hithadhoo,MV,77.94,light rain,-0.6000,73.0833,Scoop Guest House
9,Verkh-Usugli,RU,79.38,broken clouds,52.7000,115.1667,"Gostinitsa ""Sever"""
10,Seoul,KR,83.77,broken clouds,37.5683,126.9778,LOTTE HOTEL SEOUL
11,Kijang,KR,87.53,scattered clouds,35.2442,129.2139,Samyang Beach Villa
17,Katsuura,JP,82.42,scattered clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
18,Kualakapuas,ID,86.52,overcast clouds,-3.0091,114.3876,Guest House Izonk
19,Faanui,PF,77.34,few clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
20,Neiafu,TO,77.18,few clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation


In [30]:
## 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))